In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!pip install deepxde

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import deepxde as dde
from deepxde.backend import tf
from tensorflow.python.ops import math_ops
#import tensorflow_probability as tfp

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
from scipy import integrate

import imageio


In [ ]:
### CHECK IF WE ARE USING GPU ###
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# PAPERS

# https://www.mdpi.com/2073-8994/11/4/469/htm
# DEEPXDE on imaginary numbers: https://github.com/lululxvi/deepxde/issues/194
# MAZIARRAISSI https://maziarraissi.github.io/PINNs/

In [ ]:
# SEED
np.random.seed(1234)
tf.set_random_seed(1234)

In [ ]:
# Number of bits (floating points) 64 bit is way slower! Only use it when you know the model works
dde.config.real.set_float32()
tf.keras.backend.set_floatx('float32')

In [ ]:
# DOMAIN
x_interval = (-20.0, 20.0)
t_interval = (0.0, 4.0)

From $I u_t = -u_{xx} - \alpha |u|^2 u$ we rewrite it to $0=-I u_t - u_{xx} - \alpha |u|^2 u$ and substituting $u(x,t) = g(x,t) + I w(x,t)$ into the equation and finding its $a + I b$ form.

This yields: $w_t - g_{xx} - \alpha*(g^2 + w^2)*g + I (-g_t - w_{xx} - \alpha*(g^2 + w^2)*w)$

In [ ]:
alpha = 2 # alpha > 0 self focusing
          # alpha < 0 defocusing 
S = 4 # Speed of the soliton
beta = 1 # Amplitude of the soliton

In [ ]:
### DIFFERENTIAL EQUATION ###

# iu_t = -u_xx - alpha * |u|^2 * u
# iu_t + u_xx + alpha * |u|^2 * u = 0
def NLS_equation(x, y):
  # VARIABLES
  x1 = x[:,0:1]
  t = x[:,1:2]

  # FUNCTIONS
  g = y[:,0:1] # REAL
  w = y[:,1:2] # IMAG

  # REAL
  g_x = dde.grad.jacobian(y, x, i=0, j=0)
  g_t = dde.grad.jacobian(y, x, i=0, j=1)
  g_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)

  # IMAGINARY
  w_x = dde.grad.jacobian(y, x, i=1, j=0)
  w_t = dde.grad.jacobian(y, x, i=1, j=1)
  w_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)

    
  return [-w_t + g_xx + alpha * (g**2 + w**2) * g, # REAL
          g_t + w_xx + alpha *(g**2 + w**2) * w # IMAGINARY
          ]

In [ ]:
### Soliton solution function ### 

def sol_func(x):
  x1 = x[:,0:1]
  t = x[:,1:2]
  return beta * (np.sqrt(2/alpha) * np.exp(1j * (1/2 * S * x1 - 1/4 * (S**2 - beta**2) * t)) * 1/np.cosh(beta * (x1 - S * t)))

def sol_func_2(x1,t):
  return beta * (np.sqrt(2/alpha) * np.exp(1j * (1/2 * S * x1 - 1/4 * (S**2 - beta**2) * t)) * 1/np.cosh(beta * (x1 - S * t)))

def sol_func_t(x):
  x1 = x[:,0:1]
  t = x[:,1:2]
  return (15*np.sin(2*x1)*1/np.cosh(x1))/4 + 4*np.cos(2*x1)*1/np.cosh(x1)*np.tanh(x1) -1j*(15*np.cos(2*x1)*1/np.cosh(x1))/4 + 4*np.sin(2*x1)*1/np.cosh(x1)*np.tanh(x1)

plot = 0

number_steps = 5000
number_t_steps = 20

x_ax = np.linspace(x_interval[0],x_interval[1],number_steps)
t_ax = np.linspace(t_interval[0],t_interval[1],number_t_steps)

number_of_plots = number_t_steps

if(plot == 1):

  fig, ax = plt.subplots()

  for i in range(0, number_of_plots, 1):
    t_value = str(round(t_ax[i],3))

    output = sol_func_2(x_ax, t_ax[i])

    im_out = output.imag
    re_out = output.real
    abs_out = np.sqrt(output.imag**2 + output.real**2)

    ### SAVE ALL SOLUTION VALUES ###

    plt.plot(x_ax, im_out, x_ax, re_out, x_ax, abs_out, linestyle="dashed")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend(["Imag", "Real", "Abs"])
    plt.ylim([-beta, beta])
    plt.title("All solution values at: t = " + t_value)
    plt.show()

  for i in range(0, number_of_plots, 1):
    t_value = str(round(t_ax[i],3))

    output = sol_func_2(x_ax, t_ax[i])

    im_out = output.imag
    re_out = output.real
    abs_out = np.sqrt(output.imag**2 + output.real**2)

    integral_of_sol_abs_value = integrate.cumulative_trapezoid(abs_out, x_ax)

    plt.plot(x_ax, abs_out, "ro")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Integral of absolute value is: " + str(integral_of_sol_abs_value[-1]) + " at t: " + t_value)
    plt.ylim([-beta,beta])
    plt.show()


In [ ]:
### ON BOUNDARY ###

def a_bound(x, on_boundary):
  x1 = x[0]
  t = x[1]
  return on_boundary and np.isclose(x1, x_interval[0])

def b_bound(x, on_boundary):
  x1 = x[0]
  t = x[1]
  return on_boundary and np.isclose(x1, x_interval[1])

def t0_bound(x, on_initial):
  x1 = x[0]
  t = x[1]
  return on_initial or np.isclose(t, t_interval[0])

def Always_true_bound(x,_):
  x1 = x[0]
  t = x[1]
  return True

In [ ]:
### REAL CONDITIONS ###

# INITIAL CONDITIONS

def gt0_func(x):
  x1 = x[:,0:1]
  t = x[:,1:2]

  func_val = sol_func(x)

  return func_val.real

In [ ]:
### IMAGINARY CONDITIONS ###

# INITIAL CONDITIONS

def wt0_func(x):
  x1 = x[:,0:1]
  t = x[:,1:2]

  func_val = sol_func(x)

  return func_val.imag

In [ ]:
### GEOMETRY DEFINITION ###
geom_x = dde.geometry.Interval(x_interval[0],x_interval[1])
geom_time = dde.geometry.TimeDomain(t_interval[0],t_interval[1])
geomxtime = dde.geometry.GeometryXTime(geom_x,geom_time)



In [ ]:
### IMPLEMENTATION OF REAL CONDITIONS ###

# g(x,0) = f(x)
gt0_bound = dde.IC(geomxtime, gt0_func, t0_bound, component=0)


In [ ]:
### IMPLEMENTATION OF IMAGINARY CONDITIONS ###


# w(x,0) = p(x)
wt0_bound = dde.IC(geomxtime, wt0_func, t0_bound, component=1)


In [ ]:
### TEST OF COPY FUNCTION ###

g_true = dde.DirichletBC(geomxtime, gt0_func, Always_true_bound, component=0)
w_true = dde.DirichletBC(geomxtime, wt0_func, Always_true_bound, component=1)


In [ ]:
train_model = 0
restore = 1

### MODEL SETUP ###

data = dde.data.TimePDE(geomxtime, NLS_equation, [gt0_bound, wt0_bound], num_domain=5000, num_boundary=5000, num_initial=5000, train_distribution='sobol',anchors=None, solution = None, num_test=5000)

FNN_layer = [2] + [40] * 8  + [2]

net = dde.maps.FNN(FNN_layer, "tanh", "Glorot uniform")
model = dde.Model(data, net)

model.compile("adam", lr=1e-5)

if(restore == 1):
    restore_best_model_path = "model/best_model/best.ckpt-35000"
elif(restore == 0): 
    restore_best_model_path = None

if(train_model == 1):

    # CHECKPOINT

    checkpointer = dde.callbacks.ModelCheckpoint(
        "model/model.ckpt", verbose=1, save_better_only=True
    )

    # EARLY STOP

    EarlyStop = dde.callbacks.EarlyStopping(min_delta=1e-9,patience=5000,baseline=None)

    best_model_path = "model/best_model/best.ckpt"

    losshistory, train_state = model.train(epochs=50000, display_every=1000, disregard_previous_best=True, callbacks=[EarlyStop], model_restore_path = restore_best_model_path, model_save_path = best_model_path)
    model.compile("L-BFGS-B")
    
    # TRAINING AND TEST LOSS
    dde.saveplot(losshistory, train_state, issave=True, isplot=True)

elif(train_model == 0 and restore == 1):
    model.restore(restore_best_model_path, verbose=1)


In [ ]:
dpi = 300 # Resolution of the figures

filenames_real = []

for i in range(0, number_of_plots, 1):

  fig, ax = plt.subplots()

  t_value = str(round(t_ax[i],3))

  ### SAVE REAL VALUES ###

  X_model = np.vstack((x_ax, t_ax[i]*np.ones(np.size(x_ax)))).T

  output = model.predict(X_model)
  real_number = output[:,0] # g(x,t)
  imag_number = output[:,1] # w(x,t)
  abs_number = np.sqrt(real_number**2 + imag_number**2) # abs(x,t)

  filename_real = f'{i}.png'
  filenames_real.append("figures/real/real " + filename_real)

  plt.plot(x_ax, real_number)
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title("Real value at: t = " + t_value)
  plt.ylim([-beta, beta])
  plt.savefig("figures/real/real " + filename_real, dpi=dpi)
  plt.show()


In [ ]:
with imageio.get_writer("figures/real/Real.gif", mode='I') as writer:
    for filename_real in filenames_real:
        image = imageio.imread(filename_real)
        writer.append_data(image)

In [ ]:
filenames_imag = []

for i in range(0, number_of_plots, 1):

  fig, ax = plt.subplots()

  t_value = str(round(t_ax[i],3))

  ### SAVE IMAG VALUES ###

  X_model = np.vstack((x_ax, t_ax[i]*np.ones(np.size(x_ax)))).T

  output = model.predict(X_model)
  real_number = output[:,0] # g(x,t)
  imag_number = output[:,1] # w(x,t)
  abs_number = np.sqrt(real_number**2 + imag_number**2) # abs(x,t)

  filename_imag = f'{i}.png'
  filenames_imag.append("figures/imag/imag " + filename_imag)

  plt.plot(x_ax, imag_number)
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title("Imag value at: t = " + t_value)
  plt.ylim([-beta, beta])
  plt.savefig("figures/imag/imag " + filename_imag, dpi=dpi)
  plt.show()


In [ ]:
with imageio.get_writer("figures/imag/imag.gif", mode='I') as writer:
    for filename_imag in filenames_imag:
        image = imageio.imread(filename_imag)
        writer.append_data(image)

In [ ]:
filenames_abs = []

for i in range(0, number_of_plots, 1):

  fig, ax = plt.subplots()

  t_value = str(round(t_ax[i],3))

  ### SAVE ABS VALUES ###

  X_model = np.vstack((x_ax, t_ax[i]*np.ones(np.size(x_ax)))).T

  output = model.predict(X_model)
  real_number = output[:,0] # g(x,t)
  imag_number = output[:,1] # w(x,t)
  abs_number = np.sqrt(real_number**2 + imag_number**2) # abs(x,t)

  filename_abs = f'{i}.png'
  filenames_abs.append("figures/abs/abs " + filename_abs)

  plt.plot(x_ax, abs_number)
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title("Abs value at: t = " + t_value)
  plt.ylim([-beta, beta])
  plt.savefig("figures/abs/abs " + filename_abs, dpi=dpi)
  plt.show()

In [ ]:
with imageio.get_writer("figures/abs/abs.gif", mode='I') as writer:
    for filename_abs in filenames_abs:
        image = imageio.imread(filename_abs)
        writer.append_data(image)

In [ ]:
filenames_all = []

for i in range(0, number_of_plots, 1):

  fig, ax = plt.subplots()

  t_value = str(round(t_ax[i],3))

  # SAVE ONE WITH ALL VALUES #

  filename_all = f'{i}.png'
  filenames_all.append("figures/all/all " + filename_all)

  X_model = np.vstack((x_ax, t_ax[i]*np.ones(np.size(x_ax)))).T

  output = model.predict(X_model)
  real_number = output[:,0] # g(x,t)
  imag_number = output[:,1] # w(x,t)
  abs_number = np.sqrt(real_number**2 + imag_number**2) # abs(x,t)

  plt.plot(x_ax, imag_number, x_ax, real_number, x_ax, abs_number, linestyle="dashed")
  plt.legend(["Imag", "Real", "Abs"])
  plt.xlabel("x")
  plt.ylabel("y")
  plt.ylim([-beta, beta])
  plt.title("All values at: t = " + t_value)
  plt.savefig("figures/all/all " + filename_all, dpi=dpi)
  plt.show()

In [ ]:
with imageio.get_writer("figures/all/all.gif", mode='I') as writer:
    for filename_all in filenames_all:
        image = imageio.imread(filename_all)
        writer.append_data(image)

In [ ]:
### L2 RESIDUALS AND PLOTS ###

l2_real_error = []
l2_imag_error = []
l2_abs_error = []

l2_relative_real_error = []
l2_relative_imag_error = []
l2_relative_abs_error = []

for i in range(0, number_of_plots, 1):

  fig, ax = plt.subplots()

  t_value = str(round(t_ax[i],3))

  X_model = np.vstack((x_ax, t_ax[i]*np.ones(np.size(x_ax)))).T

  output = model.predict(X_model)
  real_number = output[:,0] # g(x,t)
  imag_number = output[:,1] # w(x,t)
  abs_number = np.sqrt(real_number**2 + imag_number**2) # abs(x,t)

  output_sol = sol_func(X_model)
  real_sol = np.ravel(output_sol.real)
  imag_sol = np.ravel(output_sol.imag)
  abs_sol = np.sqrt(real_sol**2 + imag_sol**2)

  real_norm = np.linalg.norm(real_number, 2)
  imag_norm = np.linalg.norm(imag_number, 2)
  abs_norm = np.linalg.norm(abs_number, 2)

  real_norm_error = np.linalg.norm(real_sol - real_number, 2)
  imag_norm_error = np.linalg.norm(imag_sol - imag_number, 2)
  abs_norm_error = np.linalg.norm(abs_sol - abs_number, 2)

  l2_real_error.append(real_norm_error)
  l2_imag_error.append(imag_norm_error)
  l2_abs_error.append(abs_norm_error)

  real_relative_norm_error = real_norm_error/real_norm
  imag_relative_norm_error = imag_norm_error/imag_norm
  abs_relative_norm_error = abs_norm_error/abs_norm

  l2_relative_real_error.append(real_relative_norm_error)
  l2_relative_imag_error.append(imag_relative_norm_error)
  l2_relative_abs_error.append(abs_relative_norm_error)

  print("\nL2-imag:", imag_norm_error,"L2-real:", real_norm_error, "L2-abs:", abs_norm_error)

  plt.plot(x_ax, imag_sol - imag_number, x_ax, real_sol - real_number, x_ax, abs_sol - abs_number, linestyle="dashed")
  plt.legend(["Imag", "Real", "Abs"])
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title("All l2 error values at: t = " + t_value)
  plt.ylim([-beta, beta])
  plt.show()

In [ ]:
plt.plot(t_ax, l2_imag_error, "--", t_ax, l2_real_error, "--", t_ax, l2_abs_error, "--")
plt.legend(["L2-imag", "L2-real", "L2-abs"])
plt.xlabel("t")
plt.ylabel("L2-error")
plt.title("L2-error of the real, imaginary and absolute value")
plt.savefig("figures/L2_error", dpi=dpi)
plt.show()

In [ ]:
plt.plot(t_ax, l2_relative_imag_error, "--", t_ax, l2_relative_real_error, "--", t_ax, l2_relative_abs_error, "--")
plt.legend(["L2-relative imag", "L2-relative real", "L2-relative abs"])
plt.xlabel("t")
plt.ylabel("Relative L2-error")
plt.title("L2-relative error of the real, imaginary and absolute value")
plt.savefig("figures/L2_relative_error", dpi=dpi)
plt.show()

In [ ]:
### CHECK THAT THE MODEL IS INVARIANT ###

l2_integral_error_app = []
l2_relative_integral_error_app = []

fig, ax = plt.subplots()

for i in range(0, number_of_plots, 1):

  t_value = str(round(t_ax[i],3))

  X_model = np.vstack((x_ax, t_ax[i]*np.ones(np.size(x_ax)))).T

  output = model.predict(X_model)
  real_number = output[:,0] # g(x,t)
  imag_number = output[:,1] # w(x,t)
  abs_number = np.sqrt(real_number**2 + imag_number**2) # abs(x,t)

  integral_of_abs_value = integrate.cumulative_trapezoid(abs_number,x_ax) # THIS SHOULD BE EQUAL TO PI

  last_abs_value = integral_of_abs_value[-1]

  l2_integral_error = np.abs(last_abs_value - np.pi)
  l2_relative_integral_error = l2_integral_error/np.abs(last_abs_value)

  l2_integral_error_app.append(l2_integral_error)
  l2_relative_integral_error_app.append(l2_relative_integral_error)

  print("\nL2 of invariance:",l2_integral_error)
  print("L2-relative error of invariance:",l2_relative_integral_error)

  plt.plot(x_ax, abs_number,"ro")
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title("Integral of absolute value is: " + str(integral_of_abs_value[-1]) + " at t:" + t_value)
  plt.ylim([-beta,beta])
  plt.show()

In [ ]:
plt.plot(t_ax, l2_integral_error_app)
plt.legend(["L2 of invariance"])
plt.xlabel("t")
plt.ylabel("L2-error of invariance")
plt.title("Absolute invariance")
plt.savefig("figures/l2_error/L2_error_invariance", dpi=dpi)
plt.show()

In [ ]:
plt.plot(t_ax, l2_relative_integral_error_app)
plt.legend(["L2-relative of invariance"])
plt.xlabel("t")
plt.ylabel("Relative L2-error of invariance")
plt.title("Relative invariance")
plt.savefig("figures/l2_error/L2_relative_error_invariance", dpi=dpi)
plt.show()